In [7]:
import tensorflow as tf
import numpy as np

In [4]:
saver=tf.train.Saver.restore

ValueError: No variables to save

In [ ]:
tf.train.AdagradOptimizer

In [5]:
24*24*1600

921600

In [20]:
np.random.randn(4,2,2)-np.random.randn(4,2,2).mean((1,2))

ValueError: operands could not be broadcast together with shapes (4,2,2) (4,) 